In [1]:
from __future__ import division, print_function, absolute_import

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [2]:
ccf_train_data = "./data/2018_4_21_22_38_train.csv"
ccf_test_data = "./data/2018_4_30_08_33_test.csv"


def load_data(filepath):
    from numpy import genfromtxt

    csv_data = genfromtxt(filepath, delimiter=",", skip_header=1)
    data = []
    labels = []

    for d in csv_data:
        data.append(d[:-1])
        labels.append(d[-1])

    return np.array(data), np.array(labels)

In [3]:
train_dataset, train_labels = load_data(ccf_train_data)
test_dataset, test_labels = load_data(ccf_test_data)

In [60]:
# Training Params
num_steps = 70000
batch_size = 100
learning_rate = 0.0002

# Network Params
image_dim = 17 # 28*28 pixels
gen_hidden_dim = 256
disc_hidden_dim = 256
noise_dim = 100 # Noise data points

# A custom initialization (see Xavier Glorot init)
def glorot_init(shape):
    return tf.random_normal(shape=shape, stddev=1. / tf.sqrt(shape[0] / 2.))

In [61]:
# Store layers weight & bias
weights = {
    'gen_hidden1': tf.Variable(glorot_init([noise_dim, gen_hidden_dim])),
    'gen_out': tf.Variable(glorot_init([gen_hidden_dim, image_dim])),
    'disc_hidden1': tf.Variable(glorot_init([image_dim, disc_hidden_dim])),
    'disc_out': tf.Variable(glorot_init([disc_hidden_dim, 1])),
}
biases = {
    'gen_hidden1': tf.Variable(tf.zeros([gen_hidden_dim])),
    'gen_out': tf.Variable(tf.zeros([image_dim])),
    'disc_hidden1': tf.Variable(tf.zeros([disc_hidden_dim])),
    'disc_out': tf.Variable(tf.zeros([1])),
}

In [62]:
# Generator
def generator(x):
    hidden_layer = tf.matmul(x, weights['gen_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['gen_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['gen_out'])
    out_layer = tf.add(out_layer, biases['gen_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer


# Discriminator
def discriminator(x):
    hidden_layer = tf.matmul(x, weights['disc_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['disc_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['disc_out'])
    out_layer = tf.add(out_layer, biases['disc_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer

# Build Networks
# Network Inputs
gen_input = tf.placeholder(tf.float32, shape=[None, noise_dim], name='input_noise')
disc_input = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input')

# Build Generator Network
gen_sample = generator(gen_input)

# Build 2 Discriminator Networks (one from noise input, one from generated samples)
disc_real = discriminator(disc_input)
disc_fake = discriminator(gen_sample)

# Build Loss
gen_loss = -tf.reduce_mean(tf.log(disc_fake))
disc_loss = -tf.reduce_mean(tf.log(disc_real) + tf.log(1. - disc_fake))

# Build Optimizers
optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)

# Training Variables for each optimizer
# By default in TensorFlow, all variables are updated by each optimizer, so we
# need to precise for each one of them the specific variables to update.
# Generator Network Variables
gen_vars = [weights['gen_hidden1'], weights['gen_out'],
            biases['gen_hidden1'], biases['gen_out']]
# Discriminator Network Variables
disc_vars = [weights['disc_hidden1'], weights['disc_out'],
            biases['disc_hidden1'], biases['disc_out']]

# Create training operations
train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [63]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [64]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)

# Training
for i in range(1, num_steps+1):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    
    batch_x, _ = next_batch(batch_size,train_dataset,train_labels)
    # Generate noise to feed to the generator
    z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])

    # Train
    feed_dict = {disc_input: batch_x, gen_input: z}
    _, _, gl, dl = sess.run([train_gen, train_disc, gen_loss, disc_loss], feed_dict=feed_dict)
    if i % 200 == 0 or i == 1:
        print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))

Step 1: Generator Loss: 0.607288, Discriminator Loss: 0.798071
Step 200: Generator Loss: 1.401344, Discriminator Loss: 0.284914
Step 400: Generator Loss: 1.259066, Discriminator Loss: 0.337371
Step 600: Generator Loss: 1.613330, Discriminator Loss: 0.222440
Step 800: Generator Loss: 2.232308, Discriminator Loss: 0.123439
Step 1000: Generator Loss: 2.319609, Discriminator Loss: 0.103510
Step 1200: Generator Loss: 2.829946, Discriminator Loss: 0.060831
Step 1400: Generator Loss: 3.336156, Discriminator Loss: 0.036222
Step 1600: Generator Loss: 3.797030, Discriminator Loss: 0.022914
Step 1800: Generator Loss: 4.000581, Discriminator Loss: 0.018566
Step 2000: Generator Loss: 4.514540, Discriminator Loss: 0.011011
Step 2200: Generator Loss: 4.457741, Discriminator Loss: 0.011719
Step 2400: Generator Loss: 4.978578, Discriminator Loss: 0.006911
Step 2600: Generator Loss: 5.380243, Discriminator Loss: 0.004618
Step 2800: Generator Loss: 5.721979, Discriminator Loss: 0.003283
Step 3000: Genera

Step 24400: Generator Loss: 18.647053, Discriminator Loss: -0.000000
Step 24600: Generator Loss: 18.761229, Discriminator Loss: -0.000000
Step 24800: Generator Loss: 18.879560, Discriminator Loss: -0.000000
Step 25000: Generator Loss: 18.988255, Discriminator Loss: -0.000000
Step 25200: Generator Loss: 19.091524, Discriminator Loss: -0.000000
Step 25400: Generator Loss: 19.205690, Discriminator Loss: -0.000000
Step 25600: Generator Loss: 19.303467, Discriminator Loss: -0.000000
Step 25800: Generator Loss: 19.397680, Discriminator Loss: -0.000000
Step 26000: Generator Loss: 19.495934, Discriminator Loss: -0.000000
Step 26200: Generator Loss: 19.585613, Discriminator Loss: -0.000000
Step 26400: Generator Loss: 19.682249, Discriminator Loss: -0.000000
Step 26600: Generator Loss: 19.782351, Discriminator Loss: -0.000000
Step 26800: Generator Loss: 19.841986, Discriminator Loss: -0.000000
Step 27000: Generator Loss: 19.958826, Discriminator Loss: 0.000000
Step 27200: Generator Loss: 20.0679

Step 48200: Generator Loss: 23.620178, Discriminator Loss: 0.000000
Step 48400: Generator Loss: 23.635944, Discriminator Loss: -0.000000
Step 48600: Generator Loss: 23.649935, Discriminator Loss: -0.000000
Step 48800: Generator Loss: 23.663416, Discriminator Loss: -0.000000
Step 49000: Generator Loss: 23.673645, Discriminator Loss: -0.000000
Step 49200: Generator Loss: 23.673777, Discriminator Loss: -0.000000
Step 49400: Generator Loss: 23.702200, Discriminator Loss: -0.000000
Step 49600: Generator Loss: 23.714205, Discriminator Loss: -0.000000
Step 49800: Generator Loss: 23.710121, Discriminator Loss: -0.000000
Step 50000: Generator Loss: 23.737734, Discriminator Loss: -0.000000
Step 50200: Generator Loss: 23.748859, Discriminator Loss: -0.000000
Step 50400: Generator Loss: 23.743643, Discriminator Loss: -0.000000
Step 50600: Generator Loss: 23.770309, Discriminator Loss: -0.000000
Step 50800: Generator Loss: 23.780508, Discriminator Loss: -0.000000
Step 51000: Generator Loss: 23.7905

In [69]:
gl

24.416204

In [70]:
dl

-0.0